In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
import emojis
import unicodedata
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


/Users/tienhuynh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tienhuynh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
address = pd.read_excel("./address.xlsx")
address

,id,addressObj/postalcode
0,239853,77019
1,1158926,77079
2,111618,77046
3,223137,77092-7307
4,223151,77054
...,...,...
1483,653244,77056-6301
1484,1642905,77003
1485,99014,77092-7312
1486,99104,77060-3507


In [3]:
review = pd.read_csv("./reviews.csv")
review

,id_x,name_x,ai_review,no of amenities,description,hotelClass,nearestMetroStations/0/distance,nearestMetroStations/1/distance,nearestMetroStations/2/distance,nearestMetroStations/3/distance,...,via_mobile,service,cleanliness,overall,value,location,sleep_quality,rooms,check_in_front_desk,business_service_(e_g_internet_access)
0,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
1,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,NaN
2,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
3,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,NaN
4,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,4.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274465,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,5.0,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN
274466,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0,3.0
274467,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,4.0,5.0,4.0,NaN,NaN,NaN,5.0,NaN,NaN
274468,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,5.0,5.0,5.0,5.0,5.0,NaN,5.0,5.0,NaN


In [4]:
review = pd.merge(left=review, right=address, left_on="id_x", right_on="id")

In [5]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274470 entries, 0 to 274469
Data columns (total 45 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   id_x                                    274470 non-null  int64  
 1   name_x                                  274470 non-null  object 
 2   ai_review                               190401 non-null  object 
 3   no of amenities                         274470 non-null  int64  
 4   description                             261606 non-null  object 
 5   hotelClass                              274470 non-null  float64
 6   nearestMetroStations/0/distance         274470 non-null  float64
 7   nearestMetroStations/1/distance         274470 non-null  float64
 8   nearestMetroStations/2/distance         274470 non-null  float64
 9   nearestMetroStations/3/distance         274470 non-null  float64
 10  nearestMetroStations/4/distance         2744

In [6]:
# Define the bins and new labels based on the hotel class categories
bins = [0, 1.5, 2.5, 3.5, 4.5, 5.0]
labels = ['Tourist', 'Standard', 'Comfort', 'First Class', 'Luxury']
review['hotelClass'] = pd.cut(review['hotelClass'], bins=bins, labels=labels, include_lowest=True)

In [7]:
metro_columns = review.columns[6:11]
review['dist_to_metro'] = review[metro_columns].mean(axis=1)

In [8]:
review['no_of_attractions'] = review['neighborhood location'].apply(
    lambda x: len(x.split(',')) if isinstance(x, str) else 0)

In [9]:
review.drop(columns=["ai_review", 
                     "photoCount", 
                     "priceRange", 
                     "rankingDenominator", 
                     "rankingPosition", 
                     "rating",
                     "roomTips/0/rating", 
                     "roomTips/0/text", 
                     "roomTips/1/rating", 
                     'roomTips/1/text', 
                     "roomTips/2/rating", 
                     "roomTips/2/text", "roomTips/3/rating", 
                     "roomTips/3/text", "roomTips/4/rating", 
                     "roomTips/4/text", "title","nearestMetroStations/0/distance", 
                     "nearestMetroStations/1/distance", "nearestMetroStations/2/distance", 
                     "nearestMetroStations/3/distance", "nearestMetroStations/4/distance", 
                     "neighborhood location", "numberOfRooms", "via_mobile", "description", "name_x"
                     ,"id"
                    ], inplace=True)

In [10]:
# Define the mapping
price_mapping = {
    '$': "low",         # Low price
    '$$': "moderate",        # Moderate price
    '$$$': "expensive",       # Expensive
    '$$$$': "very expensive"       # Very expensive
}

# Apply the mapping to the 'priceLevel' column
review['priceLevel'] = review['priceLevel'].map(price_mapping)

In [11]:
# Convert 'award' to 1/0 based on whether a value exists
review['travelerChoiceAward'] = np.where(review['travelerChoiceAward'].notna() & (review['travelerChoiceAward'] != ''), 1, 0)


In [12]:
# Function to convert CamelCase or PascalCase to snake_case
def camel_to_snake(name):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

# Apply the function to all column names
review.columns = [camel_to_snake(col) for col in review.columns]

In [13]:
review = review.rename(columns={"id_x": "id", 
                                "no of amenities": "no_of_amenities", 
                                "number_of_reviews": "no_of_reviews",
                                "traveler_choice_award": "award",
                                "text": "review",
                                "business_service_(e_g_internet_access)": "business_service",
                                "overall": "ratings",                              
                               })

In [14]:
new_order = ["id",
             "review",
             "hotel_class",
             "no_of_amenities",
             "price_level",
             "dist_to_metro",
             "no_of_attractions",
             "award",
             "no_of_reviews",
             "service",
             "cleanliness",
             "value",
             "location",
             "sleep_quality",
             "rooms",
             "check_in_front_desk",
             "business_service",
             "ratings"
            ]
review = review[new_order]

In [15]:
# Calculate the percentage of null values in each column
null_percentage = pd.DataFrame({
    'column': review.columns,
    'null_percentage': round((review.isnull().mean() * 100),2)
}).reset_index(drop=True)
null_percentage

,column,null_percentage
0,id,0.00
1,review,0.00
2,hotel_class,0.00
3,no_of_amenities,0.00
4,price_level,1.23
5,dist_to_metro,0.00
6,no_of_attractions,0.00
7,award,0.00
8,no_of_reviews,0.00
9,service,13.39


In [16]:
review['price_level'].fillna("Not Specified", inplace=True)

In [17]:
# Define columns and their respective imputation methods
columns_to_impute = {
    'cleanliness': 'median',
    'service': 'median',
    'location': 'median',
    'value': 'mean',
    'sleep_quality': 'median',
    'rooms': 'mean',
    'check_in_front_desk': 'median',
    'business_service': 'median'
}

# Impute null values using apply
for column, method in columns_to_impute.items():
    review[column] = review[column].fillna(getattr(review[column], method)())
    review[column] = review[column].round(1)

In [18]:
# Calculate the percentage of null values in each column
null_percentage = pd.DataFrame({
    'column': review.columns,
    'null_percentage': round((review.isnull().mean() * 100),2)
}).reset_index(drop=True)
null_percentage

,column,null_percentage
0,id,0.0
1,review,0.0
2,hotel_class,0.0
3,no_of_amenities,0.0
4,price_level,0.0
5,dist_to_metro,0.0
6,no_of_attractions,0.0
7,award,0.0
8,no_of_reviews,0.0
9,service,0.0


In [19]:
review = review.drop_duplicates(keep="first", subset=['review'])

In [20]:
def normalize_text(text: str, lower: bool = True) -> str:
    """
    Normalize text by lowercasing it and using the NFD method for representation.

    Args:
        text (str): comment
        lower (bool): whether to lowercase the text

    Returns:
        str: normalized comment
    """
    text = text.lower() if lower else text
    return unicodedata.normalize('NFD', text)


In [21]:
# List of columns to apply the normalization function
columns_to_normalize = ['review']

# Apply the normalization function and create new columns with the prefix "normalized_"
for column in columns_to_normalize:
    review[f'normalized_{column}'] = review[column].apply(normalize_text)

In [22]:
# Define the URL pattern
URL_PATTERN = unicodedata.normalize(
    'NFD', 
    r"(?i)\b((?:http[s]?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)"
    r"(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
)

# Define the HTML tag pattern
HTML_PATTERN = r'<[^>]+>'

def contains_url_or_html(text: str) -> int:
    """
    Check if the text contains a URL or HTML tag.

    Args:
        text (str): comment

    Returns:
        int: 1 if URL or HTML tag is present, 0 otherwise
    """
    url_flag = re.search(URL_PATTERN, text)
    html_flag = re.search(HTML_PATTERN, text)
    return int(url_flag is not None or html_flag is not None)

In [23]:
columns_contains_url = ['normalized_review']

# Apply the normalization function 
for column in columns_contains_url:
    review[f'url_{column}'] = review[column].apply(contains_url_or_html)

In [24]:
review[review['url_normalized_review']==1]

,id,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,no_of_reviews,service,cleanliness,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review,url_normalized_review
1543,113317,I'll start by saying that I prefer small hotel...,First Class,50,very expensive,172.130,5,1,7596,5.0,4.0,4.0,3.0,5.0,4.0,5.0,5.0,5.0,i'll start by saying that i prefer small hotel...,1
2116,113317,I stayed at the Hotel Casablanca for a busines...,First Class,50,very expensive,172.130,5,1,7596,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,4.0,i stayed at the hotel casablanca for a busines...,1
2490,217616,We stayed at the Muse in June during a very ho...,First Class,63,very expensive,305.644,4,1,3916,5.0,5.0,3.0,4.0,5.0,4.0,4.0,4.0,4.0,we stayed at the muse in june during a very ho...,1
4001,208454,Cannot recommend this hotel highly enough. We ...,First Class,88,very expensive,311.662,3,0,7955,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,cannot recommend this hotel highly enough. we ...,1
4421,208454,I stayed in this hotel for 5 nights at the end...,First Class,88,very expensive,311.662,3,0,7955,5.0,5.0,4.0,5.0,5.0,5.0,4.0,4.0,5.0,i stayed in this hotel for 5 nights at the end...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272056,84093,"After a lot of research, we decided on the Pal...",First Class,69,expensive,710.672,2,0,5512,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,"after a lot of research, we decided on the pal...",1
272521,84093,Great location right in down the street from D...,First Class,69,expensive,710.672,2,0,5512,5.0,5.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,great location right in down the street from d...,1
272848,84093,"Good hotel. \nRoom was well furnished, clean a...",First Class,69,expensive,710.672,2,0,5512,5.0,5.0,4.0,4.0,5.0,4.0,4.0,4.0,5.0,"good hotel. \nroom was well furnished, clean a...",1
272914,84093,We just returned from another 4 day/3 night st...,First Class,69,expensive,710.672,2,0,5512,4.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0,4.0,we just returned from another 4 day/3 night st...,1


In [25]:
def remove_html_tags_and_urls(text):
    # Remove HTML tags
    text_without_html = re.sub(r'<[^>]+>', '', text)
    # Remove URLs starting with http:// or https://
    text_cleaned = re.sub(r'http[s]?://\S+|www\.\S+', '', text_without_html)
    return text_cleaned


In [26]:
# Check if 'normalized_review' exists and apply the function
if 'normalized_review' in review.columns:
    review['normalized_review'] = review['normalized_review'].apply(remove_html_tags_and_urls)
else:
    print("The 'normalized_review' column does not exist in the DataFrame.")

In [27]:
# Pattern for English uppercase letters


def containAdvertisement(text: str) -> int:
    """
    Check if uppercase letters occupy more than half the length of the string.

    Args:
        ptext (str): review text

    Returns:
        int: 1 if uppercase letters occupy more than half the length of the string, otherwise 0
    """
    upper_pattern = r'[A-Z]'
    # Count the number of uppercase letters in the text
    upper_count = len(re.findall(upper_pattern, text))
    
    # Check if uppercase letters occupy more than half the text length
    if len(text) > 0 and upper_count > 0.5 * len(text):
        return 1
    else:
        return 0

In [28]:
review['contain_adv'] = review['normalized_review'].apply(containAdvertisement)

In [29]:
def extract_and_remove_emoji(text: str) -> (str, str):
    """
    Extract emojis from the comment and return the cleaned text without emojis.

    Args:
        text (str): comment

    Returns:
        tuple: (string containing extracted emojis, string without emojis)
    """
    # Regex pattern to match emojis
    emoji_pattern = re.compile(
        "[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F7FF"  # alchemical symbols
        u"\U0001F800-\U0001F8FF"  # geometric shapes
        u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        u"\U0001FA00-\U0001FAFF"  # chess symbols
        u"\U00002700-\U000027BF"  # dingbats
        "]+", 
        flags=re.UNICODE
    )
    
    # Find all emojis in the text
    extracted_emojis = emoji_pattern.findall(text)
    # Join the extracted emojis into a single string
    emoji_string = ' '.join(extracted_emojis)
    
    # Remove emojis from the text
    cleaned_text = emoji_pattern.sub('', text)
    
    return cleaned_text


In [30]:
review['normalized_review'] = review['normalized_review'].apply(extract_and_remove_emoji)

In [31]:
def extractEmoji(text: str) -> str:
    """
    Extract emojis from the comment.

    Args:
        text (str): comment

    Returns:
        str: a string containing the extracted emojis
    """
    # Regex pattern to match emojis
    emoji_pattern = re.compile(
        "[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F7FF"  # alchemical symbols
        u"\U0001F800-\U0001F8FF"  # geometric shapes
        u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        u"\U0001FA00-\U0001FAFF"  # chess symbols
        u"\U00002700-\U000027BF"  # dingbats
        "]+", 
        flags=re.UNICODE
    )
    
    # Find all emojis in the text
    extracted_emojis = emoji_pattern.findall(text)
    return ' '.join(extracted_emojis)

In [32]:
review['emo'] = review['normalized_review'].apply(extractEmoji)

In [33]:
review['emo'].value_counts()

emo
    274161
Name: count, dtype: int64

In [34]:
# Set seed for reproducibility
DetectorFactory.seed = 0

def is_english(text: str) -> bool:
    """
    Check if the given text is in English.

    Args:
        text (str): The text to check.

    Returns:
        bool: True if the text is in English, False otherwise.
    """
    try:
        return detect(text) == 'en'
    except:
        return False  # Return False if detection fails


In [35]:
# review['isEnglish'] = review['normalized_review'].apply(is_english)

In [36]:
# non_english_indices = review[review['isEnglish'] == False].index.tolist()

# # Step 2: Optionally save the indices list to a file (CSV, JSON, etc.)
# # For example, saving to a CSV:

# non_english_df = pd.DataFrame(non_english_indices, columns=['index'])
# non_english_df.to_csv('non_english_indices.csv', index=False) 

In [37]:
non_english = pd.read_csv("./non_english_indices.csv")

In [38]:
review = review.drop(non_english['index']).reset_index(drop=True)

In [39]:
def process_text(text):
    # Remove all special characters
    processed_feature = re.sub(r'\W', ' ', str(text))
    
    # Remove all words that include digits / numbers
    processed_feature = re.sub(r'\w*\d\w*', ' ', processed_feature)
    
    # Remove all single characters
    processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    
    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    
    # Substituting multiple spaces with a single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    
    # Return the cleaned text
    return processed_feature.strip()


columns_to_process = ['normalized_review']  
review[columns_to_process] = review[columns_to_process].applymap(process_text)


In [40]:
review

,id,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,no_of_reviews,service,...,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review,url_normalized_review,contain_adv,emo
0,113317,We had a whisper-quiet queen deluxe room couty...,First Class,50,very expensive,172.130,5,1,7596,5.0,...,5.0,5.0,5.0,4.0,4.0,5.0,we had whisper quiet queen deluxe room coutyar...,0,0,
1,113317,Great 3 night stay right after Thanksgiving. A...,First Class,50,very expensive,172.130,5,1,7596,5.0,...,5.0,5.0,4.0,4.0,4.0,5.0,great night stay right after thanksgiving agre...,0,0,
2,113317,I hesitate to give this little gem a stellar r...,First Class,50,very expensive,172.130,5,1,7596,5.0,...,5.0,5.0,5.0,4.0,4.0,5.0,i hesitate to give this little gem stellar rev...,0,0,
3,113317,You won't find a more conveniently located hot...,First Class,50,very expensive,172.130,5,1,7596,5.0,...,5.0,5.0,4.0,4.0,4.0,5.0,you won find more conveniently located hotel i...,0,0,
4,113317,This small boutique hotel right off Times Squa...,First Class,50,very expensive,172.130,5,1,7596,5.0,...,5.0,4.0,4.0,4.0,4.0,5.0,this small boutique hotel right off times squa...,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239175,235513,Beautifully decorated and comfortable rooms ev...,First Class,79,very expensive,525.276,3,1,3973,5.0,...,5.0,5.0,4.0,4.0,4.0,5.0,beautifully decorated and comfortable rooms ev...,0,0,
239176,235513,I was there with my girlfriend for 4 days at t...,First Class,79,very expensive,525.276,3,1,3973,5.0,...,5.0,5.0,4.0,4.0,4.0,5.0,i was there with my girlfriend for days at the...,0,0,
239177,235513,Upscale hotel in downtown DC. Walking distance...,First Class,79,very expensive,525.276,3,1,3973,5.0,...,5.0,5.0,4.0,4.0,4.0,4.0,upscale hotel in downtown dc walking distance ...,0,0,
239178,235513,All expectations for service and accommodation...,First Class,79,very expensive,525.276,3,1,3973,5.0,...,5.0,5.0,4.0,4.0,4.0,5.0,all expectations for service and accommodation...,0,0,


In [41]:
review.drop(columns=["url_normalized_review", 
                     "contain_adv", 
                     "emo",
                    ], inplace=True)

In [42]:
# Get the set of English stopwords
stops = set(stopwords.words('english'))

# Define a function to remove stopwords
def remove_stopwords(text):
    words = word_tokenize(text)  # Tokenize the text
    filtered_words = [word for word in words if word.lower() not in stops]  # Remove stopwords
    return ' '.join(filtered_words)  # Join the words back into a string

# Apply the function to the specified columns
columns_to_clean = ['normalized_review']
review[columns_to_clean] = review[columns_to_clean].applymap(remove_stopwords)


In [43]:
# Get the stem text
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def stem_text(text):
    
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(text)
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [44]:
# Apply the function to the specified columns
columns_to_stem = ['normalized_review']
review[columns_to_stem] = review[columns_to_stem].applymap(stem_text)

In [45]:
review.to_csv("review_clean.csv", index=False)

In [46]:
review

,id,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,no_of_reviews,service,cleanliness,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review
0,113317,We had a whisper-quiet queen deluxe room couty...,First Class,50,very expensive,172.130,5,1,7596,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,whisper quiet queen deluxe room coutyard room ...
1,113317,Great 3 night stay right after Thanksgiving. A...,First Class,50,very expensive,172.130,5,1,7596,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,great night stay right thanksgiving agree prev...
2,113317,I hesitate to give this little gem a stellar r...,First Class,50,very expensive,172.130,5,1,7596,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,hesitate give little gem stellar review fear a...
3,113317,You won't find a more conveniently located hot...,First Class,50,very expensive,172.130,5,1,7596,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,find conveniently locate hotel nyc pace count ...
4,113317,This small boutique hotel right off Times Squa...,First Class,50,very expensive,172.130,5,1,7596,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,5.0,small boutique hotel right time square perfect...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239175,235513,Beautifully decorated and comfortable rooms ev...,First Class,79,very expensive,525.276,3,1,3973,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,5.0,beautifully decorate comfortable room even tho...
239176,235513,I was there with my girlfriend for 4 days at t...,First Class,79,very expensive,525.276,3,1,3973,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,5.0,girlfriend day end march location great close ...
239177,235513,Upscale hotel in downtown DC. Walking distance...,First Class,79,very expensive,525.276,3,1,3973,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,4.0,upscale hotel downtown dc walk distance white ...
239178,235513,All expectations for service and accommodation...,First Class,79,very expensive,525.276,3,1,3973,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,5.0,expectation service accommodation exceed probl...
